In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import pandas as pd
import numpy as np

connect to db

In [2]:
sparql = SPARQLWrapper('https://dati.camera.it/sparql')

Create dataframe of the deputati from a legislation

In [3]:
dep = pd.read_csv('data/18esima_Leg/dep_18.csv')[['persona', 'nomeGruppo', 'nome', 'cognome']]
a = dep['cognome']+ ' ' + dep['nome']
dep['Parlamentare'] = a
dep.drop(['nome', 'cognome'], axis = 1, inplace= True)

a = []
for i in range(dep.shape[0]):
    a.append(dep['persona'][i][39:])

dep['id'] = a
dep.drop('persona', inplace= True, axis = 1)

p = []
dat = []
for i in dep['nomeGruppo']:
    e = i.split()
    p.append(e[-2][1:-1])
    d = e[-1].replace(')', '')
    dat.append(datetime.strptime(d[-10:], '%d.%m.%Y'))

dep['Partito'] = p
dep['dataIngresso'] = dat
dep.drop('nomeGruppo', axis = 1, inplace= True)

first_df = pd.DataFrame(columns= dep.columns)

for i in np.unique(dep['id']):
    if len(dep[dep['id'] == i]) < 2:
        first_df = pd.concat([first_df, dep[dep['id'] == i]])
    else:
        duplicate = dep[dep['id'] == i]
        first_df = pd.concat([first_df, duplicate.head(1)])


In [5]:
sparql.setReturnFormat(JSON)

leg = 18

# deputati of a specific legislation
for m in range(1,13):
   final_df = first_df.copy()
   month = f'{m:02d}'
   for d in range(1,32):
      day = f'{d:02d}'
      sparql.setQuery("""
      select distinct ?votazione ?data ?titolo ?numeroVotazione ?cognome ?nome ?espressione ?infoAssenza ?deputato where {
      ?votazione a ocd:votazione;
         dc:date '"""+'2021'+month+day+"""';
         dc:date ?data;
         rdfs:label ?titolo;
         dc:description ?descrizione;
         dc:identifier ?numeroVotazione.


      ## voti espressi
      ?voto a ocd:voto;
         ocd:rif_votazione ?votazione;
         dc:type ?espressione;
         ocd:rif_deputato ?deputato.
      OPTIONAL{?voto dc:description ?infoAssenza}


      ## info deputato
      ?deputato foaf:surname ?cognome; foaf:firstName ?nome  


      } ORDER BY ?cognome
      
      """
      )
      try:
         ret = sparql.queryAndConvert()
         res = ret["results"]["bindings"]
      except Exception as e:
         print(e)

      dep = []
      vot = []
      numero = []
      for i in res:
         dep.append((i.get('deputato')).get('value')[40:46])
         vot.append((i.get('espressione')).get('value'))
         numero.append((i.get('numeroVotazione')).get('value'))

      votazione = pd.DataFrame([dep,vot, numero], index = ['id', 'Voto1', 'numero']).T
      
      for i in list(set(votazione['numero'])):
         espressione = votazione[votazione['numero'] == i][['id', 'Voto1']]
         espressione.columns= ('id', i)
         final_df = pd.merge(final_df, espressione, how = 'left')
   
      
   final_df.to_csv('xvii-2021-'+month+'.csv', index=False)


# se numero NA di un parlamentare è maggiore 

In [66]:
dizionario_espressioni = {'Assente': '0', 'Favorevole': '1', 'Non ha votato': '0', 'Contrario': '-1', 'Astensione': '2', 'In missione': '0'}
for i in final_df.columns[4:]:
    final_df[i] = final_df[i].map(dizionario_espressioni)

final_df.fillna('0', inplace= True)
    

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [151]:
final_df.columns[4:]

Index(['634001', '635005', '635022', '635019', '635017', '635003', '635002',
       '635011', '635012', '635015',
       ...
       '645012', '645002', '645005', '645018', '645024', '645009', '645025',
       '645019', '645007', '645015'],
      dtype='object', length=160)

In [78]:
np.unique(final_df.iloc[236, 4:], return_counts=True)

(array(['0', '1'], dtype=object), array([158,   2], dtype=int64))

In [35]:
cosine_similarity(final_df[final_df.columns[4:]])

array([[1.        , 0.47688809, 0.17407766, ..., 0.        , 0.        ,
        0.        ],
       [0.47688809, 1.        , 0.14048787, ..., 0.        , 0.        ,
        0.        ],
       [0.17407766, 0.14048787, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])